# Deflection Profile Parametric Study for El Ghadioui2020
El Ghadioui2020_Structural behaviour of CFRP reinforced concrete members under bending and shear loads

In [2]:
%matplotlib widget
# %matplotlib qt

In [3]:
from bmcs_cross_section.api import MKappa, TShape, EC2, BarLayer, ReinfLayer
from bmcs_beam.api import BoundaryConditions, BoundaryConfig, DeflectionProfile, BeamDesign, LoadDeflectionParamsStudy
from os.path import join
import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
import bmcs_utils.api as bu

In [4]:
# Beam B-M-C-K1 - CFRP Bars

excel_file = join('./verification_data/el_ghadioui2020', 'fig_3.xlsx')
data = pd.read_excel(excel_file, engine='openpyxl')
ver_data = [data['B-M-C-K1_deflection'], data['B-M-C-K1_load']]

In [4]:
# Beam B-M-C-K1 - CFRP Bars

# C 50/60

E = 33525

mc = MKappa(reinforcement_type= ReinforcementType.CARBON,
            
#             concrete_material_law = ConcreteMaterialLaw.EC2,
#             f_cm = 50, # 71.6,
#             f_cd = f_ck*0.85/1.5,
            
            low_kappa = 0,
            high_kappa = 0.000082,
            E_cc = E,
            E_ct = E,
            eps_cr = 0.00006,
            eps_tu = 0.00008,
            eps_cy = 0.00175, # EC2 eps_c3
            eps_cu = 0.0035,  # EC2 eps_cu3
            mu = 0.0)

mc.cross_section_shape.B = 400
mc.cross_section_shape.H = 200
mc.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=135121, z=35, A=2 * 70.14)]

dp = DeflectionProfile(mc=mc)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
# dp.beam_design.L = 5000 # This gives perfect results
dp.F_scale = 2/1000

# bu.InteractiveWindow([dp, 
#                       dp.beam_design, 
#                       dp.mc, 
#                       dp.mc.cs_design.cross_section_shape, 
#                       dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

In [74]:
# Test f_ck
""" [
    {'f_ck': 30, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 50, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 70, 'low_kappa':-0.00001 , 'high_kappa':0.00011},
    {'f_ck': 90, 'low_kappa':-0.00002 , 'high_kappa':0.00011},
    ] """

f_ck = 90

E = EC2.get_E_cm(f_ck)

fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
f_ctd = fctk_0_05
# f_ctd = 0.85 * fctk_0_05 / 1.5
f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)


mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
            low_kappa = 0,
            high_kappa = 0.000075,
            E_cc = E,
            E_ct = E,
            eps_cr = f_ctd_fl/E,            # 0.00006,
            eps_tu = 2* f_ctd_fl/E,         # 0.00008,
            eps_cy = EC2.get_eps_c3(f_ck),
            eps_cu = EC2.get_eps_cu3(f_ck),
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=135121, z=35, A=2 * 70.14)]
# mc1.interact()


dp = DeflectionProfile(mc=mc1)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
# dp.beam_design.L = 5000 # This gives perfect results
dp.F_scale = 2/1000

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

# _, ax = plt.subplots()
# f, w = dp.get_Fw()
# ax.plot(w, f*2/1000)
# ax.plot(ver_data[0], ver_data[1])

No solution The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
No solution The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
M inverse has not succeeded, the M-Kappa solution may have failed due to a wrong kappa range or not suitable material law!
No solution The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
M inverse has not succeeded, the M-Kappa solution may have failed due to a wrong kappa range or not suitable material law!


In [13]:
# Test E

""" [
    {'E_c': 30000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_c': 35000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_c': 40000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_c': 45000, 'low_kappa':-0.00001 , 'high_kappa':0.00011 },
    ] """

E = 30000
   
mc = MKappa(low_kappa = 0,
            high_kappa = 0.00012) 
    
mc.cs_design.matrix_.trait_set(
        E_cc=E,
        E_ct=E,
        eps_cr = 0.00006,
        eps_tu = 0.00008,
        eps_cy = 0.00175,
        eps_cu = 0.0035,
        mu = 0.0,
    )  

mc.cross_section_shape_.B = 400
mc.cross_section_shape_.H = 200

bl1 = ReinfLayer(name='layer1', z=35, A=2 * 70.14, matmod='carbon')
bl1.matmod_.trait_set(E=135121, f_t=1891)
mc.cross_section_layout.add_layer(bl1)

dp = DeflectionProfile(mc=mc)
dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
dp.F_scale = 2/1000

dp.interact()

# _, ax = plt.subplots()
# f, w = dp.get_Fw()
# ax.plot(w, f*2/1000)
# ax.plot(ver_data[0], ver_data[1])

M - k recalculated


In [17]:
# saving mcs with different E

setups = [
    {'E_c': 30000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_c': 35000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_c': 40000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_c': 45000, 'low_kappa':-0.00001 , 'high_kappa':0.00011 },
    ]
mcs = []
for setup in setups:
    
    E = setup['E_c']
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],
                E_cc = E,
                E_ct = E,
                eps_cr = 0.00006,
                eps_tu = 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=135121, z=35, A=2 * 70.14)]
    mcs.append(mc1)
    

In [18]:
# running parametric study for different E
dp = DeflectionProfile(mc=mc)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
dp.F_scale = 2/1000

params_config = [
    ['mc',   dp, mcs],
]

LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C3891900>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C3BB29F0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C38195E0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C3B99590>, 
Parametric study finished.


In [21]:
# Test E_s

""" [
    {'E_s': 130000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_s': 170000, 'low_kappa':0 , 'high_kappa':0.00014 },
    {'E_s': 210000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_s': 250000, 'low_kappa':0 , 'high_kappa':0.00012 },
    ] """


E = 33525

mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,           
            low_kappa = 0,
            high_kappa = 0.00012,
            E_cc = E,
            E_ct = E,
            eps_cr = 0.00006,
            eps_tu = 0.00008,
            eps_cy = 0.00175,
            eps_cu = 0.0035,
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=250000, z=35, A=2 * 70.14)]
# mc1.interact()


dp = DeflectionProfile(mc=mc1)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
# dp.beam_design.L = 5000 # This gives perfect results
dp.F_scale = 2/1000

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

# _, ax = plt.subplots()
# f, w = dp.get_Fw()
# ax.plot(w, f*2/1000)
# ax.plot(ver_data[0], ver_data[1])

In [22]:
# saving mcs with different E_s

setups = [
    {'E_s': 130000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_s': 170000, 'low_kappa':0 , 'high_kappa':0.00014 },
    {'E_s': 210000, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'E_s': 250000, 'low_kappa':0 , 'high_kappa':0.00012 },
    ]
mcs = []
for setup in setups:
    
    E = 33525
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],
                E_cc = E,
                E_ct = E,
                eps_cr = 0.00006,
                eps_tu = 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=setup['E_s'], z=35, A=2 * 70.14)]
    mcs.append(mc1)
    

In [23]:
# running parametric study for different E_s
dp = DeflectionProfile(mc=mc)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
dp.F_scale = 2/1000

params_config = [
    ['mc',   dp, mcs],
]

LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C5C07220>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C00DC0E0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C8302D60>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C5E9E720>, 
Parametric study finished.


In [19]:
# Test carbon strength

""" [
    {'eps_sy': 1800/135121, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'eps_sy': 2200/135121, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'eps_sy': 2600/135121, 'low_kappa':0 , 'high_kappa':0.00014 },
    {'eps_sy': 3000/135121, 'low_kappa':0 , 'high_kappa':0.00015 },
    ] """

E = 33525

mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,           
            low_kappa = 0,
            high_kappa = 0.00012,
            E_cc = E,
            E_ct = E,
            eps_cr = 0.00006,
            eps_tu = 0.00008,
            eps_cy = 0.00175,
            eps_cu = 0.0035,
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=3000/135121, E=135121, z=35, A=2 * 70.14)]
# mc1.interact()


dp = DeflectionProfile(mc=mc1)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
# dp.beam_design.L = 5000 # This gives perfect results
dp.F_scale = 2/1000

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

# _, ax = plt.subplots()
# f, w = dp.get_Fw()
# ax.plot(w, f*2/1000)
# ax.plot(ver_data[0], ver_data[1])

No solution The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.


In [24]:
# saving mcs with different carbon strength

setups = [
    {'eps_sy': 1800/135121, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'eps_sy': 2200/135121, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'eps_sy': 2600/135121, 'low_kappa':0 , 'high_kappa':0.00014 },
    {'eps_sy': 3000/135121, 'low_kappa':0 , 'high_kappa':0.00015 },
    ]
mcs = []
for setup in setups:
    
    E = 33525
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],
                E_cc = E,
                E_ct = E,
                eps_cr = 0.00006,
                eps_tu = 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=setup['eps_sy'], E=135121, z=35, A=2 * 70.14)]
    mcs.append(mc1)
    

In [25]:
# running parametric study for different carbon strength
dp = DeflectionProfile(mc=mc)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
dp.F_scale = 2/1000

params_config = [
    ['mc',   dp, mcs],
]

LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C81B1130>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C38A3400>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C82649A0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x00000205C81413B0>, 
Parametric study finished.


In [9]:
# Test concrete compressive strength

""" [
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 30, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 50, 'low_kappa':0 , 'high_kappa':0.0001 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 70, 'low_kappa':-0.00001 , 'high_kappa':0.00013 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 90, 'low_kappa':-0.000002 , 'high_kappa':0.00012 },
    ] """

E = 33525
   
mc = MKappa(low_kappa = -0.000002,
            high_kappa = 0.00012, n_kappa = 500,) 
    
mc.cs_design.matrix='EC2 with plateau'
mc.cs_design.matrix_.trait_set(
        E_cc=E,
        E_ct=E,
        eps_cr = 0.00006,
        eps_tu = 0.00008,
        eps_cy = 0.00175,
        eps_cu = 0.0035,
        mu = 0.0,
        f_cd = 90,
    )  

mc.cross_section_shape_.B = 400
mc.cross_section_shape_.H = 200

bl1 = ReinfLayer(name='layer1', z=35, A=2 * 70.14, matmod='carbon')
bl1.matmod_.trait_set(E=135121, f_t=1891)
mc.cross_section_layout.add_layer(bl1)

dp = DeflectionProfile(mc=mc)
dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
dp.F_scale = 2/1000

dp.interact()

M - k recalculated


In [15]:
# saving mcs with different concrete compressive strength

setups = [
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 30, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 50, 'low_kappa':0 , 'high_kappa':0.0001 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 70, 'low_kappa':-0.00001 , 'high_kappa':0.00013 },
    {'concrete_material_law': ConcreteMaterialLaw.EC2, 'f_cd': 90, 'low_kappa':-0.000002 , 'high_kappa':0.00012 },
    ]
mcs = []
for setup in setups:
    
    E = 33525
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],
                 
                 n_kappa = 400,
                 
                 concrete_material_law = setup['concrete_material_law'],
                 f_cd = setup['f_cd'],
                 
                E_cc = E,
                E_ct = E,
                eps_cr = 0.00006,
                eps_tu = 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=135121, z=35, A=2 * 70.14)]
    mcs.append(mc1)
    

In [16]:
# running parametric study for different concrete compressive strength
dp = DeflectionProfile(mc=mc, n_load_steps=251)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
dp.F_scale = 2/1000

params_config = [
    ['mc',   dp, mcs],
]

LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000001C4CA5B75E0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000001C4CA5407C0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000001C4CA641360>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x000001C4CA5F5B80>, 
Parametric study finished.


In [42]:
# Test concrete tensile strength

""" [
    {'f_ck': 0, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 30, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 50, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 70, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 90, 'low_kappa':0 , 'high_kappa':0.00012 },
    ] """

E = 33525

f_ck = 90

fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
f_ctd = fctk_0_05
# f_ctd = 0.85 * fctk_0_05 / 1.5
f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)
print(f_ctd_fl/E)
mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,   
             
            n_kappa = 1000,
             
            low_kappa = 0,
            high_kappa = 0.00012,
            E_cc = E,
            E_ct = E,
            eps_cr = f_ctd_fl/E,            # 0.00006,
            eps_tu = 2* f_ctd_fl/E,         # 0.00008,
            eps_cy = 0.00175,
            eps_cu = 0.0035,
            mu = 0.0)

mc1.cross_section_shape.B = 400
mc1.cross_section_shape.H = 200
mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=135121, z=35, A=2 * 70.14)]
# mc1.interact()


dp = DeflectionProfile(mc=mc1)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
# dp.beam_design.L = 5000 # This gives perfect results
dp.F_scale = 2/1000

# mc1.interact()

bu.InteractiveWindow([dp, 
                      dp.beam_design, 
                      dp.mc, 
                      dp.mc.cs_design.cross_section_shape, 
                      dp.mc.cs_design.cross_section_layout], tight_layout=True).interact()

# _, ax = plt.subplots()
# f, w = dp.get_Fw()
# ax.plot(w, f*2/1000)
# ax.plot(ver_data[0], ver_data[1])

0.00014746443096998806


In [14]:
f_ck = 90

fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
f_ctd = fctk_0_05
# f_ctd = 0.85 * fctk_0_05 / 1.5
f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)
f_ctd_fl

4.94374504826885

In [5]:
# saving mcs with different concrete tensile strength

setups = [
    {'f_ck': 0, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 30, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 50, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 70, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 90, 'low_kappa':0 , 'high_kappa':0.00012 },
    ]

mcs = []
for setup in setups:
    
    E = 33525
    
    f_ck = setup['f_ck']

    fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
    f_ctd = fctk_0_05
    # f_ctd = 0.85 * fctk_0_05 / 1.5
    f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)
    
    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                 
                n_kappa = 1000,
                 
                low_kappa = setup['low_kappa'],
                high_kappa = setup['high_kappa'],                
                E_cc = E,
                E_ct = E,
                eps_cr = f_ctd_fl/E,            # 0.00006,
                eps_tu = 2* f_ctd_fl/E,         # 0.00008,
                eps_cy = 0.00175,
                eps_cu = 0.0035,
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=135121, z=35, A=2 * 70.14)]
    mcs.append(mc1)
    

In [6]:
# running parametric study for different concrete tensile strength
dp = DeflectionProfile(mc=mc, n_load_steps=31)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
dp.F_scale = 2/1000

params_config = [
    ['mc',   dp, mcs],
]

LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000020CF04BD7C0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000020CF044D130>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000020CED8204A0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000020CEFCE04F0>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000020CEFE70400>, 
Parametric study finished.


In [5]:
# saving mcs with different f_ck

f_ck_setups = [
    {'f_ck': 30, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 50, 'low_kappa':0 , 'high_kappa':0.00012 },
    {'f_ck': 70, 'low_kappa':-0.00001 , 'high_kappa':0.00011},
    {'f_ck': 90, 'low_kappa':-0.00002 , 'high_kappa':0.00011},
    ]
mcs = []
for f_ck_setup in f_ck_setups:
    f_ck = f_ck_setup['f_ck']
    
    E = EC2.get_E_cm(f_ck)
    
    fctk_0_05 = EC2.get_f_ctk_0_05(f_ck)
    f_ctd = fctk_0_05
    # f_ctd = 0.85 * fctk_0_05 / 1.5
    f_ctd_fl = max((1.6 - 200 /1000) * f_ctd, f_ctd)

    mc1 = MKappa(reinforcement_type= ReinforcementType.CARBON,
                low_kappa = f_ck_setup['low_kappa'],
                high_kappa = f_ck_setup['high_kappa'],
                E_cc = E,
                E_ct = E,
                eps_cr = f_ctd_fl/E,            # 0.00006,
                eps_tu = 2* f_ctd_fl/E,         # 0.00008,
                eps_cy = EC2.get_eps_c3(f_ck),
                eps_cu = EC2.get_eps_cu3(f_ck), # EC2 eps_cu3
                mu = 0.0)
    mc1.cross_section_shape.B = 400
    mc1.cross_section_shape.H = 200
    mc1.cross_section_layout.reinforcement = [Reinforcement(eps_sy=0.014, E=135121, z=35, A=2 * 70.14)]
    mcs.append(mc1)
    

In [6]:
dp = DeflectionProfile(mc=mc)

dp.beam_design.beam_conf_name = BoundaryConfig.FOUR_PB
dp.beam_design.beam_conf_name.first_load_distance = 1925
dp.beam_design.L = 4700
dp.F_scale = 2/1000

params_config = [
    ['mc',      dp, mcs],
#         ['L',      dp.beam_design, [5000]],
#         ['B',      mc.cross_section_shape, [300, 500]],
#         ['H',      mc.cross_section_shape, [21505, 300]],
#         ['E_ct',   mc, [33525, 34000]],
#         ['E_cc',   mc, [33525, 34000]],
#         ['eps_cr', mc, [0.00003, 0.00005, 0.00006]],
#         ['eps_cy', mc, [0.0015, 0.00175, 0.002, 0.0025]],
#         ['eps_cu', mc, [0.002, 0.0025, 0.0035, 0.004]],
#         ['eps_tu', mc, [0.00004, 0.00005, 0.00006]],
#         ['mu',     mc, [0, 1]],
#         ['z',      mc.cross_section_layout.reinforcement[0], [1, 29]],
#         ['A',      mc.cross_section_layout.reinforcement[0], [70.14, 2* 70.14]],
#         ['E',      mc.cross_section_layout.reinforcement[0], [130000, 135121, 140000]],
#         ['eps_sy', mc.cross_section_layout.reinforcement[0], [0.010, 0.014, 0.018]],
#         ['n_x',    dp.beam_design, [10, 1000]],
]

LoadDeflectionParamsStudy(dp).run(params_config, ver_data)

Parametric study is running...
mc: <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000024AA11DCF90>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000024AA1251860>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000024AA12FD630>, <bmcs_cross_section.mkappa.mkappa.MKappa object at 0x0000024AA12FD9F0>, 
Parametric study finished.
